Take a look at the [repository](https://github.com/davexhardware/CycleGAN-VAE-NST) for more information

# Installation

Always remember to pull the repo

In [ ]:
!git pull

If needed, create a virtual environment for the experiments

In [ ]:
import os
import sys

# Define the name of the virtual environment
venv_name = 'venv'

# Create the virtual environment
os.system(f'{sys.executable} -m venv {venv_name}')

print(f'Virtual environment "{venv_name}" created successfully.')

Install the required packages to the virtual environment

In [ ]:
%pip install -r requirements.txt

# Datasets

Use your own dataset by creating the appropriate folders and adding in the images.

-   Create a dataset folder under `/dataset` for your dataset.
-   We will create subfolders `testA`, `testB`, `trainA`, and `trainB` under your dataset's folder. For our experiment, we're going to extract the tensors from the images available in online datasets, the <a href="https://www.kaggle.com/datasets/jessicali9530/celeba-dataset">Celeba dataset</a> and the <a href="https://www.kaggle.com/datasets/ibrahimserouis99/one-piece-image-classifier">>Onepiece Dataset</a> you can download with:

\*\***Notice that you have to change folders before running those scripts**\*\*

In [ ]:
%python ./data/custom_dataset_nst/prepare_celeba.py
%python ./data/custom_dataset_nst/prepare_onepiece.py

This will download and flatten the datasets, but the Onepiece contains a lot of images showing actions, movements and scenes that must be excluded or cropped. In order to do this we used a simple Computer Vision classic method, the *Haarcascades*, but we also reference a more complex approach, based on a pretrained Face Recognition DL model. You can check them in [crop_onepiece.py](./data/custom_dataset_nst/crop_onepiece.py) and [crop_onepiece_deep_.py](./data/custom_dataset_nst/crop_onepiece_deep.py)

In [ ]:
%python ./data/custom_dataset_nst/crop_onepiece.py

Now you can convert the images to tensors using this script, that will use the default DataLoader with some additional features to convert and store images in the folders you will specify inside tensors in new folders that will be renamed adding '_pt' at the end.

It will also resize the image to a squared \<LOAD_SIZE\>px and crop to \<CROP\>px.

\*\***Also here you will have to change directories and check the command usage**\*\*

Change gpu_ids value if you want to run it on GPU.

In [ ]:
%python ./iterate_dataset.py --transform_float16 --preprocess resize_and_crop --dataroot ./datasets --gpu_ids <GPU_ID> --load_size <LOAD_SIZE> --crop_size <CROP>

Now you can pick up the number of tensors that you want and move them to the training folders.

In [ ]:
#I want to pick N random files from source_dir and move them to dest_dir

import os
import random
import shutil

def move_files(dset):
    files = os.listdir(dset['source_dir'])

    if len(files) < dset['num_files_to_move']:
        print(f"Warning: Only {len(files)} files found in {dset['source_dir']}. Moving all of them.")
        num_files_to_move = len(files)
    else:
        num_files_to_move = dset['num_files_to_move']
    
    random_files = random.sample(files, num_files_to_move)

    # Move the files to the destination directory
    for file in random_files:
        source_path = os.path.join(dset['source_dir'], file)
        dest_path = os.path.join(dset['dest_dir'], file)
        shutil.copy(source_path, dest_path)

    print(f"Copied {num_files_to_move} random files from {dset['source_dir']} to {dset['dest_dir']}")


# Define source and destination directories
dirs=[
    {
        'source_dir' : './datasets/img_align_celeba_pt',
        'dest_dir' : './datasets/trainA',
        'num_files_to_move' : 2000
    },
    {
        'source_dir' : './datasets/onepiece_pt',
        'dest_dir' : './datasets/trainB', 
        'num_files_to_move' : 2000
    }
]

# Ensure that the source directories exist
for el in dirs:
    if not os.path.exists(el['source_dir']):
        raise Exception(f"Source directory {el['source_dir']} doesn't exist")


# Create the destination directories if they don't exist and move the files
for el in dirs:
    os.makedirs(el['dest_dir'], exist_ok=True)
    move_files(el)


If you want to verify the effective sizes of the set, use this (eventually add test folders)

In [ ]:
!ls ./datasets/trainB  | wc -l
!ls ./datasets/trainA  | wc -l

2000
2000


# Training

Ensure that CUDA is available on your machine, otherwise you should run the model on the CPU or download the necessary CUDA drivers and compatible <a href="https://pytorch.org/">PyTorch</a> version

In [ ]:

import torch

print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("cuDNN enabled:", torch.backends.cudnn.enabled)
print("cuDNN version:", torch.backends.cudnn.version() if torch.backends.cudnn.is_available() else "Not available")
print("CUDA Device Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU detected")

x = torch.randn(1, 1, 32, 32, device="cuda")  # Random tensor on GPU
conv = torch.nn.Conv2d(1, 1, kernel_size=3).cuda()  # Simple Conv2D layer
with torch.backends.cudnn.flags(enabled=True):
    output = conv(x)
print("cuDNN acceleration is working!" if torch.backends.cudnn.enabled else "cuDNN is not being used.")

So now it's time to train the model. In order to set up the correct parameters, please take a closer look at the [options](./options/base_options.py) files and to the [tips](./docs/tips.md)

If you have a GUI and/or an access to the ports of the machine you are training on, you may enjoy looking at some training loss curves and intermediate results on the visdom-provided dashboard. 
First, remove  `--display_id -1`  from the next command, and when you start it the script will connect automatically, if  `python -m visdom.server` is running.

In [ ]:
%python train.py --dataroot ./datasets --name portraits2op --lambda_identity 0 --latent_dim 256 --init_type xavier --norm bn --netG ResnetKVAE --dataset_mode tensor --crop_size 128 --batch_size 16 --verbose --n_epochs 50 --n_epochs_decay 50 --display_id -1 --beta1 0.65 --lambda_A 9.0 --lambda_kl 1.1 --lr 1e-4

### Visualize

In case you don't have the possibility to use visdom and you want to manually check the results during training epochs


In [ ]:
%pip install matplotlib

In [ ]:
%ls ./checkpoints/portraits2op/web/images/

In [ ]:
import matplotlib.pyplot as plt

img = plt.imread('./checkpoints/portraits2op/web/images/epoch002_fake_B.png')
plt.imshow(img)

img = plt.imread('./checkpoints/portraits2op/web/images/epoch002_real_A.png')
plt.imshow(img)

### While training
In case you have huge models or architectures and limited disk space, use the following script to clear the older checkpoints of the model. You can run it also during the training.

In [ ]:
import re

checkpoints_dir='./checkpoints/portraits2op'

def remove_files_with_regex(directory, pattern):
    for filename in os.listdir(directory):
        if re.match(pattern, filename):
            os.remove(os.path.join(directory, filename))

pattern = r'[0-9]+\_net\_[GD]\_[AB]\.pth'
remove_files_with_regex(checkpoints_dir, pattern)

# Testing

Once your model has trained, copy over the last checkpoint to a format that the testing model can automatically detect:

Use:
- `cp ./checkpoints/<MODELNAME>/latest_net_G_A.pth ./checkpoints/<MODELNAME>/latest_net_G.pth` if you want to transform images from class A to class B 
- or`cp ./checkpoints/<MODELNAME>/latest_net_G_B.pth ./checkpoints/<MODELNAME>/latest_net_G.pth` if you want to transform images from class B to class A.

In case you want to test models at different epochs, create a new *cp_test* folder and one subfolder for each training epoch you want to test, e.g. `cp ./checkpoints/real2onepiece_portraits_pt/250_net_G_A.pth ./cp_test/real2op250/latest_net_G.pth`
Testing here is done using images, so you'll have to load them in the testA folder.
If you want to pick a defined number of images from the train starting folder to the test one, run this script:

In [11]:
import os
import random
import shutil
def move_to_test(img_dir, test_dir, num_files):
  if os.path.exists(img_dir):
    images = [f for f in os.listdir(img_dir) if os.path.isfile(os.path.join(img_dir, f))]
    random.shuffle(images)
    images_to_move = images[:num_files]
    if not os.path.exists(test_dir):
        os.mkdir(test_dir)
    for image in images_to_move:
        if '_fake' in image:
            shutil.copy(os.path.join(img_dir, image), os.path.join(test_dir, image.replace('_fake','')))
  else:
    print(f"Directory {img_dir} does not exist")

path= './results/inverse_results/real2op200_reference_idt_full_B_to_A/test_latest/images'
pathdst = './datasets/test_fakeportr2op/testA'
test_size = 100
move_to_test(path, pathdst, test_size)

In order to let the model generate the images, run:

-   `python test.py --dataroot datasets/<EXPERIMENT>/testA --name <MODEL_NAME> --model test --no_dropout`

Change the `--dataroot` and `--name` to be consistent with your trained model's configuration.

> from https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix:
> The option --model test is used for generating results of CycleGAN only for one side. This option will automatically set --dataset_mode single, which only loads the images from one set. On the contrary, using --model cycle_gan requires loading and generating results in both directions, which is sometimes unnecessary. The results will be saved at ./results/. Use --results_dir {directory_path_to_save_result} to specify the results directory.

**For your own experiments, you might want to specify --netG, --norm, --no_dropout to match the generator architecture of the trained model.**

In [ ]:
!python test.py --dataroot datasets/horse2zebra/testA --name horse2zebra_pretrained --model test --no_dropout

In [ ]:
!python test.py --dataroot ./datasets/test_real2op --checkpoints_dir ./cp_test --name real2op200_reference_idt_full_B_to_A --crop_size 128 --load_size 140 --netG resnet_9blocks --no_dropout --norm batch